In [1]:
import pdb, math
import matplotlib.pyplot as plt
import numpy as np
import torch
import pdb
plt.rcParams["figure.figsize"] = (12,12)

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
class SDF(nn.Module):
    def __init__(self):
        super(SDF, self).__init__()
        self.fc1 = nn.Linear(3, 512)
        self.drop = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 512)
        self.fc5 = nn.Linear(512, 512)     
        self.fc6 = nn.Linear(512, 1)
        
    def forward(self, x):
#         pdb.set_trace()
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [4]:
import numpy as np
coords = np.load('coords.npy')
SDFs = np.load('SDF.npy')

In [5]:
idxes = [i for i, s in enumerate(SDFs) if s<20]

In [6]:
from torch.utils.data import Dataset, DataLoader

class SDFDataset(Dataset):

    def __init__(self,coords,SDFs):
       
        self.coords = coords
        self.SDFs = SDFs

    def __len__(self):
        return len(self.SDFs)

    def __getitem__(self, idx):
        
        coord = torch.from_numpy(self.coords[idx])
        sdf = torch.from_numpy(np.array(self.SDFs[idx])).unsqueeze(0)
#         pdb.set_trace()
        sample = {'coords': coord, 'sdf': sdf}

        return sample

In [7]:
import random
from torch.utils import data
dataset = SDFDataset(coords,SDFs)

idx = list(range(len(coords)))
random.shuffle(idx)  # in-place shuffle the indices to facilitate random splitting
train_idx = idx[:int(len(coords)*0.8)]
test_idx = idx[int(len(coords)*0.8):]

train_set = data.Subset(dataset, train_idx)
test_set = data.Subset(dataset, test_idx)

train_dataloader = DataLoader(train_set, batch_size=512,
                        shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_set, batch_size=512,
                        shuffle=True, num_workers=4)

In [8]:
len(coords)*0.8

392256.80000000005

In [9]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, sample in enumerate(train_loader):
#         pdb.set_trace()
        data, target = sample['coords'].to(device), sample['sdf'].to(device)
        optimizer.zero_grad()
        output = model(data)
#         pdb.set_trace()
        
        loss = F.l1_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, device, test_loader, optimizer, epoch, best_loss=1):
    model.eval()
    losses = []
    for batch_idx, sample in enumerate(test_loader):
        data, target = sample['coords'].to(device), sample['sdf'].to(device)
        output = model(data)
        loss = F.l1_loss(output, target)
        losses.append(loss)
#     pdb.set_trace()
    loss = sum(losses)/len(losses)
    print(f'test loss:{loss}')
    if loss<best_loss:
        torch.save(model.state_dict(), 'sdf.torch')
        best_loss = loss
    return best_loss

In [10]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
model = SDF().to(device)

In [11]:
optimizer = optim.Adam(model.parameters(), lr=0.00001)
best_loss = 1

for epoch in range(1, 50 + 1):
    train(model, device, train_dataloader, optimizer, epoch)
    best_loss = test(model, device, test_dataloader, optimizer, epoch, best_loss)

Train Epoch: 1 [0/392256 (0%)]	Loss: 6.488205
Train Epoch: 1 [51200/392256 (13%)]	Loss: 5.590211
Train Epoch: 1 [102400/392256 (26%)]	Loss: 1.074756
Train Epoch: 1 [153600/392256 (39%)]	Loss: 1.003680
Train Epoch: 1 [204800/392256 (52%)]	Loss: 0.785843
Train Epoch: 1 [256000/392256 (65%)]	Loss: 0.672025
Train Epoch: 1 [307200/392256 (78%)]	Loss: 0.586334
Train Epoch: 1 [358400/392256 (91%)]	Loss: 0.464577
test loss:0.3255421817302704
Train Epoch: 2 [0/392256 (0%)]	Loss: 0.305795
Train Epoch: 2 [51200/392256 (13%)]	Loss: 0.160061
Train Epoch: 2 [102400/392256 (26%)]	Loss: 0.118831
Train Epoch: 2 [153600/392256 (39%)]	Loss: 0.117776
Train Epoch: 2 [204800/392256 (52%)]	Loss: 0.059922
Train Epoch: 2 [256000/392256 (65%)]	Loss: 0.072693
Train Epoch: 2 [307200/392256 (78%)]	Loss: 0.051957
Train Epoch: 2 [358400/392256 (91%)]	Loss: 0.065199
test loss:0.04084573686122894
Train Epoch: 3 [0/392256 (0%)]	Loss: 0.040038
Train Epoch: 3 [51200/392256 (13%)]	Loss: 0.042856
Train Epoch: 3 [102400/392

In [20]:
model.load_state_dict(torch.load('SDF.torch'))

<All keys matched successfully>

In [25]:
import numpy as np
model = model.eval()
model(torch.from_numpy(np.array([2.3454492 , -0.96645486,  8.859472])).unsqueeze(0).float())

tensor([[4.0132]], grad_fn=<AddmmBackward>)

In [22]:
for i, x in enumerate(SDFs):
    if x<-0.2: break

In [23]:
SDFs[i]

4.0081353

In [24]:
coords[i]

array([ 2.3454492 , -0.96645486,  8.859472  ], dtype=float32)